In [1]:
import gspread, logging
account = gspread.service_account("credentials.json")
spreadsheet = account.open("FairDivision")
spreadsheet

<Spreadsheet 'FairDivision' id:1tJPV-y-r1TAx5FqbrqecKPJMeKHTtIDeiYck8eLoGKY>

In [2]:
import input
input.logger.addHandler(logging.StreamHandler())
input.logger.setLevel(logging.INFO)

rows = input.read_rows(spreadsheet)
agents, items, entitlement_normalized_preferences  = input.analyze_rows(rows)

Rows: 1000, Cols: 29
items: ['חוץ', 'בטחון', 'אוצר', 'בט"פ', 'משפטים', 'פנים', 'בריאות', 'חינוך', 'וועדת הכספים']
agents: ['הליכוד', 'הציונות הדתית', 'ש"ס', 'אגודת ישראל']
entitlements: {'הליכוד': 32, 'הציונות הדתית': 14, 'ש"ס': 11, 'אגודת ישראל': 7}, total: 64.000000
raw_preferences: 
	הליכוד:		{'חוץ': 20.0, 'בטחון': 20.0, 'אוצר': 20.0, 'בט"פ': 10.0, 'משפטים': 10.0, 'פנים': 10.0, 'בריאות': 10.0, 'חינוך': 20.0, 'וועדת הכספים': 20.0}		140.000000
	הציונות הדתית:		{'חוץ': 10.0, 'בטחון': 20.0, 'אוצר': 10.0, 'בט"פ': 30.0, 'משפטים': 20.0, 'פנים': 10.0, 'בריאות': 20.0, 'חינוך': 20.0, 'וועדת הכספים': 10.0}		150.000000
	ש"ס:		{'חוץ': 5.0, 'בטחון': 5.0, 'אוצר': 20.0, 'בט"פ': 5.0, 'משפטים': 10.0, 'פנים': 30.0, 'בריאות': 20.0, 'חינוך': 20.0, 'וועדת הכספים': 25.0}		140.000000
	אגודת ישראל:		{'חוץ': 5.0, 'בטחון': 5.0, 'אוצר': 5.0, 'בט"פ': 5.0, 'משפטים': 5.0, 'פנים': 10.0, 'בריאות': 10.0, 'חינוך': 20.0, 'וועדת הכספים': 40.0}		105.000000
normalized_preferences: 
	הליכוד:		{'חוץ': 9.143, 'בטחון': 9.143

In [3]:
import allocate
allocate.logger.addHandler(logging.StreamHandler())
allocate.logger.setLevel(logging.INFO)

map_agent_to_fractions = allocate.allocate(agents, entitlement_normalized_preferences)

leximin allocation: הליכוד gets { 100.0% of חוץ, 100.0% of בטחון, 100.0% of אוצר, 13.634% of משפטים, 76.675% of בריאות, 100.0% of חינוך, 58.632% of וועדת הכספים} with value 1.44.
הציונות הדתית gets { 100.0% of בט"פ, 86.366% of משפטים} with value 1.44.
ש"ס gets { 100.0% of פנים, 23.325% of בריאות} with value 1.44.
אגודת ישראל gets { 41.368% of וועדת הכספים} with value 1.44.

bounded sharing allocation: הליכוד gets { 100.0% of חוץ, 100.0% of בטחון, 100.0% of אוצר, 13.634% of משפטים, 76.675% of בריאות, 100.0% of חינוך, 58.632% of וועדת הכספים} with value 1.44.
הציונות הדתית gets { 100.0% of בט"פ, 86.366% of משפטים} with value 1.44.
ש"ס gets { 100.0% of פנים, 23.325% of בריאות} with value 1.44.
אגודת ישראל gets { 41.368% of וועדת הכספים} with value 1.44.

map_agent_to_fractions: {'הליכוד': array([1.   , 1.   , 1.   , 0.   , 0.136, 0.   , 0.767, 1.   , 0.586]), 'הציונות הדתית': array([0.   , 0.   , 0.   , 1.   , 0.864, 0.   , 0.   , 0.   , 0.   ]), 'ש"ס': array([0.   , 0.   , 0.   , 0.   , 

In [10]:
import output
import importlib
importlib.reload(output)

new_row_count = len(agents)+2
new_col_count = len(items)+5
output_sheet = output.worksheet(spreadsheet, new_row_count, new_col_count)
output_sheet

# input_range = input.range(1, 1, len(rows), len(rows[0]))
# output.update_cells(input_range)

output version 2


<Worksheet 'output' id:236568254>

In [11]:
output_sheet.clear()

{'spreadsheetId': '1tJPV-y-r1TAx5FqbrqecKPJMeKHTtIDeiYck8eLoGKY',
 'clearedRange': 'output!A1:O1001'}

In [12]:
# Insert headings:
new_cells = output.cells(rows, agents, items, map_agent_to_fractions)
# new_cells = []
# new_cells += [gspread.Cell(i+1, 1, "=input!"+gspread.utils.rowcol_to_a1(i+1,1)) for i in range(len(rows))]  # Copy column 1
# new_cells += [gspread.Cell(i+1, 2, "=input!"+gspread.utils.rowcol_to_a1(i+1,2)) for i in range(len(rows))]  # Copy column 2
output_sheet.update_cells(new_cells, value_input_option='USER_ENTERED')

הליכוד :  [1.    1.    1.    0.    0.136 0.    0.767 1.    0.586]
הציונות הדתית :  [0.    0.    0.    1.    0.864 0.    0.    0.    0.   ]
ש"ס :  [0.    0.    0.    0.    0.    1.    0.233 0.    0.   ]
אגודת ישראל :  [0.    0.    0.    0.    0.    0.    0.    0.    0.414]


{'spreadsheetId': '1tJPV-y-r1TAx5FqbrqecKPJMeKHTtIDeiYck8eLoGKY',
 'updatedRange': 'output!A1:O6',
 'updatedRows': 6,
 'updatedColumns': 14,
 'updatedCells': 81}

In [12]:
# Change the cell format:
first_cell = gspread.utils.rowcol_to_a1(2, 3)
last_cell = gspread.utils.rowcol_to_a1(len(agents)+2, len(items)+5)
output_sheet.format(f"{first_cell}:{last_cell}", {"numberFormat": {"type": "PERCENT", "pattern": "##.#%"}})

{'spreadsheetId': '1tJPV-y-r1TAx5FqbrqecKPJMeKHTtIDeiYck8eLoGKY',
 'replies': [{}]}